# Boston house-prices prediction with decision tree

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import itertools
import sys
from sklearn.datasets import load_boston

sys.path.append("..")
from models.decision_tree import DecisionTreeRegressor
from models.linear_regression import LinearRegression
from models.ensemble import BaggingRegressor, GradientBoostingRegressor
from utils.visualization import plot_decision_boundary

In [2]:
%matplotlib inline

np.random.seed(1)

## Load the dataset

The Boston house-prices prediction dataset housing values in suburbs of Boston, each instance is described with 14 attributes, 13 of them are use for prediction, the remaining one, MEDV(Median value of owner occupied homes in $1000's) is the target.

In [3]:
# Load an partition the data
(X, y) = load_boston(return_X_y=True)

perm = np.random.permutation(X.shape[0])
pivot = int(X.shape[0]*0.7)
x_train, y_train = X[:pivot, :], y[:pivot]
x_test, y_test = X[pivot:, :], y[pivot:]

## Define a baseline

To asses the quality of the model we may compare its perfromance against that of a simple baseline model, for instance Linear Regression.

In [4]:
# Create and fit baseline model
baseline = LinearRegression()
baseline.fit(x_train, y_train)

# Compute metrics on training and test set
rmse_train = np.sqrt(np.mean(np.square(baseline.predict(x_train)-y_train)))
y_hat = baseline.predict(x_test)
rmse = np.sqrt(np.mean(np.square(y_hat-y_test)))

print("RMSE on the train set: %.2f" % rmse_train)
print("RMSE on the validation set: %.2f" % rmse)

RMSE on the train set: 3.00
RMSE on the validation set: 23.35


So now our benchmark to beat is a RMSE of 23.39, anything performing worse than that it's not worth it since the simple baseline is able to beat it.

To fit a decision tree we have to do some hyperparameter selection. We have to set a maximum depth and a minimum impurity. Let's first start with an educated guess to asses the model selection.

In [5]:
# Create and fit a tree
tree = DecisionTreeRegressor(max_depth=6, min_impurity=1)
tree.fit(X=x_train, y=y_train)
rmse_train = np.sqrt(np.mean(np.square(tree.predict(x_train)-y_train)))

y_hat = tree.predict(x_test)
rmse = np.sqrt(np.mean(np.square(y_hat-y_test)))

print("RMSE on the train set: %.2f" % rmse_train)
print("RMSE on the validation set: %.2f" % rmse)

RMSE on the train set: 1.97
RMSE on the validation set: 6.92


## Improving the model

The new model is clearly able to outperform the baseline. But maybe we can improve it with hyperparameter search. Due to the small size of the dataset the size of the validation set might not be enough to discriminate with confidence between similar models performance. We may use cross-validation to overcome this problem.

### Cross-validation

In [6]:
def cross_validate(model, k):
    X_fold = X
    y_fold = y
    pivot = int(X_fold.shape[0]/k)
    cum_rmse = 0
    for _ in range(k):
        # Always take firt fold as test
        x_test, y_test = X_fold[:pivot, :], y_fold[:pivot]
        x_train, y_train = X_fold[pivot:, :], y_fold[pivot:]
        
        # fit the model
        model.fit(X=x_train, y=y_train)
        y_hat = model.predict(x_test)
        
        cum_rmse += np.sqrt(np.mean(np.square(y_hat-y_test)))
        
        X_fold = np.concatenate((x_train, x_test))
        y_fold = np.concatenate((y_train, y_test))
        
    return cum_rmse/k

### Grid search over hyperparameter selection

Iterate over possible combinations of hyperparameter values to find the better one.

In [7]:
# Initialize best tree with the previous one
best_tree = tree
best_rmse = rmse
best_hyperparams = (6, 1)

# Define grid search
depths = [4, 6, 8]
impurities = [0.01, 0.1, 1]

for depth, impurity in itertools.product(depths, impurities):
    curr_tree = DecisionTreeRegressor(max_depth=depth, min_impurity=impurity)
    curr_rmse = cross_validate(curr_tree, 5)
    if curr_rmse < best_rmse:
        print("Found better hyperparameter selection, depth={}, impuriy={}. New rmse: {}".format(depth, impurity, curr_rmse))
        best_tree = curr_tree
        best_rmse = curr_rmse
        best_hyperparams = (depth, impurity)

Found better hyperparameter selection, depth=4, impuriy=0.01. New rmse: 5.823268478476467
Found better hyperparameter selection, depth=6, impuriy=0.01. New rmse: 5.464568660983798


## Ensemble

### Bagging
To improve the model further we can use a bunch of trees instead of a single one. Each tree is trained on a slightly different dataset. This datasets are random samples with replacement of the original dataset. This is known as the bootstrap, a technique known to reduce the variance of the model.

In [8]:
# Create the model
base_tree = DecisionTreeRegressor(max_depth=best_hyperparams[0], min_impurity=best_hyperparams[1])
bag_trees = BaggingRegressor(base_tree, n_models=50)

# Validate
rmse = cross_validate(bag_trees, 5)
print("Cross-Validation rmse: %.2f" % rmse)

Cross-Validation rmse: 4.64


### Random Forest

The idea behind bagging is that a bunch of simple but uncorrelated classifiers will tend to agree on their correct predictions but answer more randomly on their incorrect ones. However if a small subset of the features is highly correlated with the output then trees in the bag will tend to based their decision on this subset of features leading to more correlated trees. Random forest brings the idea of bagging into the trees with what is sometimes called "feature bagging". At each split the tree looks only into some random subset of the features, uncorrelating the trees. The size of this subset is also a hyperparameter.

In [9]:
# Create the model specifying `p`, size of the subset of features to consider at each split
base_tree = DecisionTreeRegressor(max_depth=best_hyperparams[0], min_impurity=best_hyperparams[1], p=0.5)
random_forest = BaggingRegressor(base_tree, n_models=50)

# Validate
rmse = cross_validate(random_forest, 5)
print("Cross-Validation rmse: %.2f" % rmse)

Cross-Validation rmse: 4.37


### Gradient boosting
Learn to predict the errors of the previous model.

In [ ]:
# Create the model
base_tree = DecisionTreeRegressor(max_depth=3)
boosted_trees = GradientBoostingRegressor(base_tree, learning_rate=0.05, max_models=500)

# Validate
rmse = cross_validate(boosted_trees, 5)
print("Cross-Validation rmse: %.2f" % rmse)